In [ ]:
from dorna2 import Dorna

In [1]:
###############
# Config Cell #
###############

robot_ip_address = "localhost"  # robot ip address
output = [0, 0, 1]  # output pin for the suction cup(pin, off state, on state)
tcp = [0, 0, 0, 0, 0, 0]  # define the tool length and position
speed = 0.5  # speed of the robot

sleep_period = 1 # period allowing other robot to react to see if pin 5=0 and to set its pin 5=0

pin_move_in_progress = 5  # pin we use to signal a move is in progress
pin_move_positions = [1, 2, 3, 4]  # pin we use to signal placment position

grab_positions = [[x, y, z, a, b, c],  # position to grab first chip
                  [x, y, z, a, b, c],  # position to grab second chip
                  [x, y, z, a, b, c],  # position to grab third chip
                  [x, y, z, a, b, c],  # position to grab fourth chip
                  [x, y, z, a, b, c]]  # position to grab fifth chip

place_positions = [[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],  # Top row
                    [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], # middle row 
                    [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]] # bottom row

middle_pose = [0, 0, 0, 0, 0, 0]# here is the safe postion for robot between picking and placing
safe_joint = [1, 1, 1, 1 ,1 ,1 ]# safe position so other robot can work

NameError: name 'x' is not defined

In [2]:
# tic-tac-toe board matrix
board = [[None, None, None],
         [None, None, None],
         [None, None, None]]

# setting the placement positions on the board to corresponding to pin values
# (row, col): (pin 0, pin 1, pin 2, pin 3)
position_to_pins = {
    (0, 0): [0, 0, 0, 0],  # Top-left
    (0, 1): [0, 0, 0, 1],  # Top-middle
    (0, 2): [0, 0, 1, 0],  # Top-right
    (1, 0): [0, 1, 0, 0],  # Middle-left
    (1, 1): [0, 1, 0, 1],  # Center
    (1, 2): [0, 1, 1, 0],  # Middle-right
    (2, 0): [1, 0, 0, 0],  # Bottom-left
    (2, 1): [1, 0, 0, 1],  # Bottom-middle
    (2, 2): [1, 0, 1, 0]   # Bottom-right
}

NameError: name 'Dorna' is not defined

In [ ]:
robot = Dorna() # create the Dorna object

In [ ]:
robot.connect(robot_ip_address)  # connects to robot

robot.set_motor(1)  # turns on the robot motors
robot.jmove(safe_pose)  #set safe initial position
robot.set_output(output[0], output[1]) # set output, (pin, state)

In [ ]:
def set_pins(pins, values):  # sets IO pins
    for pin, value in zip(pins, values): # iterates pairs of pins and value
        robot.set_output(pin, value) # sets each pin to corresponding value

def read_pins(pins):  # reads IO pins
    return [robot.get_input(pin) for pin in pins] # returns of a list of input states or value for each pin

def update_board(board, row, col, player):  # updates the board matrix
    board[row][col] = player # set player marker in board matrix

def check_win(board, player):  # checks for a win
    win_conditions = [
        # Check rows
        [board[0][0], board[0][1], board[0][2]],
        [board[1][0], board[1][1], board[1][2]],
        [board[2][0], board[2][1], board[2][2]],
        # Check columns
        [board[0][0], board[1][0], board[2][0]],
        [board[0][1], board[1][1], board[2][1]],
        [board[0][2], board[1][2], board[2][2]],
        # Check diagonals
        [board[0][0], board[1][1], board[2][2]],
        [board[2][0], board[1][1], board[0][2]]
    ]
    return [player, player, player] in win_conditions # checks if a win condition is met

def perform_move(robot, grab_position, place_position, move_position_pins):
    # executes the robots turn
    robot.pick_n_place(
        pick_pose=grab_position, # pose from which the robot grabs the chip
        middle_pose=middle_pose, # pose between pick and place
        place_pose=place_position, # pose where the robot places the chip
        end_pose=safe_pose, # clear pose move to after placing the chip
        tcp=tcp, # tool length and position (tool center point)
        speed=speed # speed of movement
    )

    set_pins(PIN_MOVE_POSITIONS, move_position_pins) # set the move position using io pins

    set_pins([PIN_MOVE_IN_PROGRESS], [0]) # signal move completion


    robot.sleep(SLEEP_PERIOD) #  sleep to allow the other robot to read the move

def find_open_position(board):  # finds an open position on the board
    for row in range(3): # iterates over rows
        for col in range(3): # iterates over colums
            if board[row][col] is None: # checks if position is empty
                return (row, col) # return empty position
    return None # return if no empty positions are avalible

# game loop #
grab_index = 0  # start with the first grab position

for move_number in range(9):  # max 9 moves in a Tic Tac Toe game

    set_pins([PIN_MOVE_IN_PROGRESS], [1]) # signal that this robot is making a move

    position = find_open_position(board) # select a random open position on the board
    if position:
        row, col = position # get row and colum of opne position
        move_position_pins = POSITION_TO_PINS[(row, col)] # get corresponding pins

        # update the board matrix
        player = "X" if move_number % 2 == 0 else "O"  # alternate between players
        update_board(board, row, col, player) # update board matrix

        # perform the move using the current grab position
        perform_move(robot, grab_position=GRAB_POSITIONS[grab_index],
                     place_position=PLACE_POSITIONS[row * 3 + col],
                     move_position_pins=move_position_pins)

        # update the grab position index
        grab_index = (grab_index + 1) % len(GRAB_POSITIONS)  # cycle through grab positions

        # check for a win condition
        if check_win(board, player):
            print(f"Player {player} wins!")
            # move to safe position and exit loop
            robot.jmove(safe_pose)
            break # exit loop if the player wins
    else:
          # no open position found, end the game
          print("It's a draw!")
          robot.jmove(safe_pose)  # move to safe position
          break # exit loop if there is a draw

    # signal that this robot has completed the move
    set_pins([PIN_MOVE_IN_PROGRESS], [0])

    # wait for the other robot's turn to complete
    while read_pins([PIN_MOVE_IN_PROGRESS])[0] == 1:
        robot.sleep(SLEEP_PERIOD)  # waiting interval

# Close connection
robot.close()